# Analyse de la Pyramide des Âges des Élus Conseillers Municipaux

La présente analyse se concentre sur l'exploration du Répertoire National des Élus, en particulier sur les données relatives aux conseillers municipaux en France. Les données utilisées proviennent du fichier `elus-conseillers-municipaux-cm.csv` disponible sur [data.gouv.fr](https://static.data.gouv.fr/resources/repertoire-national-des-elus-1/20250613-142603/elus-conseillers-municipaux-cm.csv). Ce répertoire contient des informations détaillées sur les élus municipaux, notamment leur identité, fonction, date de naissance, catégorie socio-professionnelle, etc.

L'objectif de cette analyse est de générer une pyramide des âges par tranche de 5 ans pour les hommes et les femmes, en excluant les personnes de plus de 100 ans et en fournissant un libellé de tranche explicite. Cette visualisation permettra d'appréhender la structure démographique des élus municipaux et d'identifier les tendances en termes de répartition par âge et par sexe.


## Méthodologie

La méthodologie employée pour cette analyse repose sur l'utilisation de requêtes SQL via DuckDB pour traiter les données et préparer le dataset nécessaire à la visualisation. Les principales étapes consistent à calculer l'âge des élus à partir de leur date de naissance, à répartir ces âges en tranches de 5 ans, puis à compter les effectifs d'hommes et de femmes dans chaque tranche d'âge. Les données sont ensuite traitées avec Pandas pour restructurer le dataset et préparer la visualisation. Enfin, la bibliothèque Altair est utilisée pour créer la pyramide des âges sous forme de graphique à barres.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [2]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://static.data.gouv.fr/resources/repertoire-national-des-elus-1/20250613-142603/elus-conseillers-municipaux-cm.csv", safe_mode=True)
print("✅ Données chargées avec succès")

✅ Données chargées avec succès


## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [3]:
# Exécution de la requête
df = run_query(""" WITH age_calc AS (
  SELECT 
    "Code sexe" AS sexe,
    CAST(SUBSTRING("Date de naissance", 7, 4) AS INTEGER) AS year_birth
  FROM loaded_dataset
),
age_tranche AS (
  SELECT 
    sexe,
    FLOOR((2024 - year_birth) / 5) * 5 AS age_tranche
  FROM age_calc
  WHERE 2024 - year_birth BETWEEN 0 AND 100
),
tranche_label AS (
  SELECT 
    sexe,
    age_tranche,
    CONCAT(CAST(age_tranche AS VARCHAR), '-', CAST(age_tranche + 4 AS VARCHAR)) AS label_tranche
  FROM age_tranche
)
SELECT 
  label_tranche AS tranche_age,
  SUM(CASE WHEN sexe = 'M' THEN 1 ELSE 0 END) AS hommes,
  SUM(CASE WHEN sexe = 'F' THEN 1 ELSE 0 END) AS femmes
FROM tranche_label
GROUP BY label_tranche
ORDER BY MIN(age_tranche) """)
print(f"Résultats : {len(df)} lignes")
df.head()

Résultats : 17 lignes


,tranche_age,hommes,femmes
0,15.0-19.0,14.0,6.0
1,20.0-24.0,947.0,608.0
2,25.0-29.0,3958.0,2313.0
3,30.0-34.0,8478.0,6017.0
4,35.0-39.0,16546.0,13735.0


## 📈 Visualisation

La bibliothèque principale utilisée est Altair, qui est une bibliothèque de visualisation de données Python très performante. Altair est adaptée pour créer des visualisations interactives et personnalisables, comme cette pyramide des âges, qui nécessite une représentation graphique claire et lisible. Altair permet de créer des graphiques complexes de manière simple et intuitive.

In [4]:
import pandas as pd
import duckdb as ddb
import altair as alt
import pandas as pd

df['tranche_age'] = df['tranche_age'].astype(str)
df['order'] = df['tranche_age'].str.split('-').str[0].str.replace('.0', '').astype(float)

df_filtered = df[df['tranche_age'] != '95.0-99.0'].copy()

df_long = pd.melt(
    df_filtered,
    id_vars=['tranche_age', 'order'],
    value_vars=['hommes', 'femmes'],
    var_name='genre',
    value_name='nombre'
)

df_long['signe'] = df_long['genre'].map({'hommes': -1, 'femmes': 1})
df_long['nombre_signe'] = df_long['signe'] * df_long['nombre']

dataviz = alt.Chart(df_long).mark_bar(
    cornerRadiusEnd=2,
    stroke='#f0f0f0',
    strokeWidth=1
).encode(
    x=alt.X(
        'nombre_signe:Q',
        title='Population',
        axis=alt.Axis(format='s')
    ),
    y=alt.Y(
        'tranche_age:N',
        sort=alt.EncodingSortField('order', order='descending'),
        title=None,
        axis=alt.Axis(labelFontSize=12)
    ),
    color=alt.Color(
        'genre:N',
        scale=alt.Scale(
            domain=['hommes', 'femmes'],
            range=['#1f77b4', '#d62728']
        ),
        legend=alt.Legend(title='Genre')
    ),
    tooltip=[
        alt.Tooltip('genre:N', title='Genre'),
        alt.Tooltip('tranche_age:N', title='Tranche d\'âge'),
        alt.Tooltip('nombre:Q', format=',', title='Effectif')
    ]
).properties(
    title='Pyramide des âges des élus conseillers municipaux',
    width=500,
    height=350
).configure_axis(
    grid=False,
    tickSize=0
).configure_view(
    stroke=None
).configure_title(
    fontSize=16,
    anchor='start',
    color='#333'
)
dataviz

alt.Chart(...)

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*

In [5]:

# --- Variables injectées par le script ---
FINAL_OBJECT_VARIABLE_NAME = 'dataviz'
OUTPUT_IMAGE_NAME = 'published/notebooks/duckit_analysis_20250809_223002.png'
OUTPUT_HTML_NAME = 'published/notebooks/duckit_analysis_20250809_223002.html'

# ===================================================================
# CELLULE INJECTÉE AUTOMATIQUEMENT (VERSION ROBUSTE)
# ===================================================================
import sys
import os
# On importe les modules nécessaires pour l'export au cas où
try:
    from bokeh.io import save as bokeh_save
except ImportError:
    bokeh_save = None

try:
    # On s'assure que le dossier de sortie existe
    output_dir = os.path.dirname(OUTPUT_IMAGE_NAME)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    # On utilise globals().get() pour une récupération plus sûre
    final_object = globals().get(FINAL_OBJECT_VARIABLE_NAME)

    if final_object is None:
        # On lève une NameError pour être cohérent avec le code original
        raise NameError(f"name '{FINAL_OBJECT_VARIABLE_NAME}' is not defined")

    print(f"INFO: Variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée. Tentative d'exportation...")

    object_type = str(type(final_object))

    if 'plotly.graph_objs._figure.Figure' in object_type:
        print(f"--> Détecté : Plotly. Sauvegarde HTML et PNG.")
        # 1. Sauvegarde HTML pour l'interactivité
        print(f"--> Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.write_html(OUTPUT_HTML_NAME, include_plotlyjs='cdn')
        # 2. Sauvegarde PNG pour l'aperçu statique
        try:
            print(f"--> Tentative de sauvegarde PNG directe dans : {OUTPUT_IMAGE_NAME}")
            final_object.write_image(OUTPUT_IMAGE_NAME, scale=3, width=1200, height=800)
            print(f"--> Image Plotly sauvegardée avec succès.")
        except Exception as e:
            print(f"AVERTISSESEMENT: La sauvegarde directe en PNG a échoué (kaleido est-il installé?). L'image statique ne sera pas générée.", file=sys.stderr)
            print(f"   Erreur: {e}", file=sys.stderr)
    elif 'folium.folium.Map' in object_type:
        print(f"--> Détecté : Folium. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur pour que le script de post-traitement sache qu'il s'agit de Folium
        with open(f"{OUTPUT_HTML_NAME}.is_folium", "w") as f:
            f.write("true")
    elif 'altair.vegalite' in object_type and hasattr(final_object, 'save'):
        print(f"--> Détecté : Altair. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
    elif 'bokeh.plotting' in object_type and bokeh_save is not None:
        print(f"--> Détecté : Bokeh. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        bokeh_save(final_object, filename=OUTPUT_HTML_NAME, title="")
    elif 'matplotlib.figure.Figure' in object_type:
        print(f"--> Détecté : Matplotlib. Sauvegarde dans : {OUTPUT_IMAGE_NAME}")
        final_object.savefig(OUTPUT_IMAGE_NAME, dpi=300, bbox_inches='tight')
    else:
        print(f"AVERTISSEMENT: Type non supporté : {object_type}", file=sys.stderr)
except NameError:
    print(f"AVERTISSEMENT: Aucune variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée.", file=sys.stderr)
except Exception as e:
    print(f"ERREUR lors de l'exportation : {e}", file=sys.stderr)


INFO: Variable 'dataviz' trouvée. Tentative d'exportation...
--> Détecté : Altair. Sauvegarde HTML dans : published/notebooks/duckit_analysis_20250809_223002.html
